# Демич Александр Андреевич Мит-101

## Установка необходимых зависимостей

In [11]:
!pip install pandas
!pip install scikit-learn
!pip install plotly
!pip install ipywidgets

## Загрузка данных из файла

In [32]:
file_path = './SMSSpamCollection'
with open(file_path, 'r', encoding='utf-8') as file:
    data = file.readlines()

### Обработка данных из файла

Этот блок кода предназначен для загрузки и предобработки данных из текстового файла, который содержит сообщения и их принадлежность.

* Метод `strip()` в Python используется для удаления пробельных символов (пробелы, табуляции, переносы строк) с начала и конца строки.  Он возвращает новую строку без этих символов, не изменяя исходную.

* Метод `split()` в Python используется для разделения строки на список подстрок (слов) по заданному разделителю.  Он возвращает новый список строк, не изменяя исходную строку.

* Символ `\t` в Python представляет собой **символ табуляции**. Он используются для представления специальных символов в строках. Он использован в `data` для разграничения данных

После загрузки данных и создания DataFrame, я использовал метод `replace()` для перевода английских обозначений классов `'ham'` и `'spam'` на русские эквиваленты `'не спам'` и `'спам'` соответственно. Это сделано для улучшения читаемости и понимания меток классов в дальнейшем анализе.

In [33]:
import pandas as pd
import plotly.express as px

# Преобразование данных в DataFrame и перевод
df = pd.DataFrame([line.strip().split('\t') for line in data], columns=['Признак', 'Сообщение'])
df.replace('ham', "не спам", inplace=True)
df.replace('spam', "спам", inplace=True)

# Вывод гистограммы
fig_hist = px.histogram(df, x ="Признак", title="Количество спама/не спама в dataset" )
fig_hist.show()

### CountVectorizer

Это инструмент, предназначенный для векторизации текста. Он преобразует текстовые данные в числовой формат, который можно использовать для обучения моделей машинного обучения.

Большинство алгоритмов машинного обучения работают с числовыми данными, а не с текстом. Поэтому, когда мы имеем дело с текстовыми данными, их необходимо предварительно обработать и преобразовать в числовую форму.

#### Как работает `CountVectorizer`
1.  **Токенизация:**
    *   `CountVectorizer` разбивает текст на отдельные слова или токены.
    *   По умолчанию он использует пробелы для разделения слов, но это поведение можно настроить.

2.  **Построение словаря (vocabulary):**
    *   `CountVectorizer` создает словарь, который содержит все уникальные слова, найденные во входных текстах.
    *   Словарь представляет собой список уникальных слов.

3.  **Преобразование текста в векторы:**
    *   Для каждого документа (текста) `CountVectorizer` создает вектор.
    *   Каждый элемент этого вектора соответствует слову из словаря, а значение элемента — это частота появления этого слова в данном документе.
    
#### Зачем нужен `CountVectorizer`

*   **Преобразование текста в числовой формат:** Делает текст пригодным для обучения моделей машинного обучения.
*   **Представление текста в виде признаков:** Каждое слово (точнее, частота его появления) становится признаком для модели.
*   **Упрощение работы с текстом:** Автоматизирует процесс токенизации и векторизации.
*   **Подготовка данных для классификации:** Необходимый шаг в задачах классификации текста (спам, анализ тональности и т.д.).

In [34]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
# Перевод меток в числовой формат
df['Признак'] = df['Признак'].map({'не спам': 0, 'спам': 1})

# Разделение данных на признаки и метки
X = df['Сообщение']
y = df['Признак']

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Векторизация текста
vectorizer = CountVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

## Теорема Байеса

Теорема Байеса — это фундаментальная концепция в теории вероятностей, которая описывает, как обновлять вероятность события на основе новых данных. Она выражается следующей формулой:

$$P(A|B) = \frac{P(B|A) \cdot P(A)}{P(B)}$$

Где:

*   $P(A|B)$ — **апостериорная вероятность** события A при условии, что произошло событие B. Это вероятность, которую мы хотим вычислить.
*   $P(B|A)$ — **правдоподобие** события B при условии, что произошло событие A.
*   $P(A)$ — **априорная вероятность** события A. Это вероятность события A до получения новых данных (события B).
*   $P(B)$ — **вероятность** события B. Она используется для нормализации и в итоге не влияет на выбор класса с наибольшей вероятностью.

## MultinomialNB
Наивный байесовский классификатор (MultinomialNB) использует Теорему Байеса для оценки вероятности того, что образец данных принадлежит к определенному классу. При этом предполагается, что признаки являются независимыми.

Классифицируем текст (спам/не спам):

*   A - это класс (спам/не спам).
*   B - это текст (слова в документе).

## BernoulliNB
Bernoulli Naive Bayes — это еще один вид наивного байесовского классификатора, который, в отличие от MultinomialNB, подходит для задач, где признаки представлены в бинарном (двоичном) формате. Это означает, что для каждого признака (например, слова в тексте) значение может быть либо 0 (признак отсутствует), либо 1 (признак присутствует). BernoulliNB особенно полезен в задачах классификации текста, где важен факт наличия слова в документе, а не его частота.

BernoulliNB, как и другие наивные байесовские классификаторы, предполагает, что **признаки независимы друг от друга** при условии известного класса. Это упрощение, которое часто хорошо работает на практике, особенно в текстовых задачах.

Главное отличие от MultinomialNB в том, что BernoulliNB обрабатывает **бинарные признаки**. Это означает, что для каждого слова в словаре мы не используем частоту его появления, а просто отмечаем: присутствует ли это слово в документе (1) или нет (0).

**Сравнение с MultinomialNB:**

| Особенность       | BernoulliNB                                       | MultinomialNB                      |
|-------------------|---------------------------------------------------|------------------------------------|
| **Тип признаков** | Бинарные (0 или 1)                                | Частоты слов                       |
| **Использование** | Фактическое наличие слова                         | Частота появления слова            |
| **Подходит для**  | Задачи, где важен факт наличия                    | Задачи, где важна частота          |
| **Размер данных** | Обычно не очень требователен                      | Может работать с большими объемами |
| **Обучение**      | Подсчет вероятностей присутствия/отсутствия слова | Подсчет частот слов                |

## Logistic Regression (Логистическая Регрессия)

Логистическая регрессия — это алгоритм машинного обучения, используемый для решения задач **бинарной классификации**, где нужно предсказать вероятность принадлежности объекта к одному из двух классов. Несмотря на название "регрессия", это **классификационный алгоритм**, а не регрессионный.

Как и линейная регрессия, логистическая регрессия строит линейную модель на основе признаков (входных данных).  Модель имеет вид:

$$z = w_0 + w_1x_1 + w_2x_2 + ... + w_nx_n$$

   *   $z$ - это линейная комбинация входных признаков.
   *   $x_1, x_2, ..., x_n$ - это признаки (входные данные).
   *   $w_0, w_1, ..., w_n$ - это веса или коэффициенты модели. $w_0$ - это свободный член (bias).
   
Линейная комбинация $z$ преобразуется с помощью сигмоидной функции (логистической функции):

$$ \sigma(z) = \frac{1}{1 + e^{-z}} $$

   *   Сигмоидная функция имеет S-образную форму и принимает значения от 0 до 1.
   *   Она интерпретирует линейную комбинацию `z` как вероятность принадлежности к одному из классов.

## Ключевые различия

| Характеристика         | Логистическая Регрессия                                   | Наивный Байесовский Классификатор                                 |
|------------------------|-----------------------------------------------------------|-------------------------------------------------------------------|
| **Тип классификации**  | Дискриминативный                                          | Генеративный                                                      |
| **Основной принцип**   | Поиск границы между классами                              | Моделирование распределения признаков в каждом классе             |
| **Предположения**      | Линейная разделимость, слабая корреляция между признаками | Независимость признаков                                           |
| **Оптимизация**        | Минимизация функции потерь                                | Подсчет и использование вероятностей                              |
| **Скорость обучения**  | Относительно более медленный чем наивный Баейс            | Быстрее, особенно для больших наборов данных                      |
| **Интерпретируемость** | Высокая                                                   | Сложнее интерпретировать                                          |
| **Тип данных**         | Подходит для различных типов данных                       | Особенно хорошо для текстовых данных (MultinomialNB, BernoulliNB) |

# LinearSVC (Линейный Классификатор на основе метода опорных векторов)

`LinearSVC` — это алгоритм машинного обучения, который используется для задач **бинарной и мультиклассовой классификации**. Он является разновидностью метода опорных векторов (Support Vector Machine, SVM) и особенно хорошо подходит для задач, где данные могут быть разделены линейной границей.

LinearSVC` ищет **линейную гиперплоскость**, которая наилучшим образом разделяет данные. В двумерном пространстве это будет прямая линия, в трехмерном — плоскость, а в многомерном — гиперплоскость.

  *   Уравнение гиперплоскости: $$ w_0 + w_1x_1 + w_2x_2 + ... + w_nx_n = 0 $$

  *   $x_1, x_2, ..., x_n$ — это признаки (входные данные).
  *   $w_0, w_1, ..., w_n$ — это веса или коэффициенты, определяющие положение и ориентацию гиперплоскости. $w_0$ - это свободный член (bias).
    
**Отличия от Логистической Регрессии:**

| Характеристика                      | LinearSVC                                            | Логистическая Регрессия                                                             |
|-------------------------------------|------------------------------------------------------|-------------------------------------------------------------------------------------|
| **Основная цель**                   | Максимизация зазора между классами                   | Поиск границы, которая максимально увеличивает вероятность правильной классификации |
| **Функция потерь**                  | `hinge loss`                                         | Log loss (cross-entropy)                                                            |
| **Вероятность**                     | Не предоставляет вероятности напрямую                | Предоставляет вероятности                                                           |
| **Чувствительность к выбросам**     | Менее чувствителен                                   | Более чувствительна                                                                 |
| **Регуляризация**                   | Обычно использует L2-регуляризацию                   | Использует регуляризацию L1 и L2                                                    |
| **Решение для мультиклассификации** | "Один против остальных" (`one-vs-rest`)              | Может использовать `softmax` для множества классов                                  |
| **Применение**                      | Линейно разделимые данные, когда важен большой зазор | Различные задачи классификации, когда важны вероятности                             |

**Отличия от Наивных Байесовских Классификаторов:**

| Характеристика        | LinearSVC                                                                | Наивный Байесовский Классификатор                                 |
|-----------------------|--------------------------------------------------------------------------|-------------------------------------------------------------------|
| **Подход**            | Дискриминативный                                                         | Генеративный                                                      |
| **Предположения**     | Линейная разделимость, не делает предположений о независимости признаков | Независимость признаков                                           |
| **Тип данных**        | Различные типы данных                                                    | Особенно хорошо для текстовых данных (MultinomialNB, BernoulliNB) |
| **Скорость обучения** | Может быть медленнее наивного Байеса                                     | Обычно быстрее                                                    |
| **Интерпретация**     | Интерпретировать веса сложнее                                            | Интерпретировать вероятности признаков легче                      |

# GradientBoostingClassifier (Градиентный Бустинг)

`GradientBoostingClassifier` — это алгоритм машинного обучения, использующий **ансамбль (композицию) слабых моделей** (обычно решающих деревьев) для решения задач классификации. Он является разновидностью техники **градиентного бустинга**, которая последовательно строит модели, исправляя ошибки предыдущих.

**Градиентный спуск:** На каждом шаге обучения используется градиентный спуск для минимизации функции потерь $L(y, F(x))$.

**Функция потерь (Loss function):** Функция, которая показывает, насколько хорошо модель предсказывает значения. Например, для задачи классификации может использоваться cross-entropy loss или логистическая функция потерь.

   Пример функции cross-entropy loss:
$$ L(y, p) = -[y \log(p) + (1-y) \log(1-p)] $$ 
   *  где $y$ - истинное значение класса (0 или 1)
   *  $p$ - предсказанная вероятность принадлежности к классу 1
   
**Отличия от Логистической Регрессии:**

| Характеристика     | `GradientBoostingClassifier`               | Логистическая Регрессия                       |
|--------------------|--------------------------------------------|-----------------------------------------------|
| **Тип модели**     | Ансамблевый метод, на основе деревьев      | Линейная модель                               |
| **Подход**         | Нелинейный                                 | Линейный                                      |
| **Обучение**       | Последовательное (бустинг)                 | Оптимизация весов одновременно                |
| **Функция потерь** | Гибкая (зависит от задачи)                 | Обычно log loss (cross-entropy)               |
| **Интерпретация**  | Менее интерпретируема                      | Более интерпретируема                         |
| **Применение**     | Работает с линейными и нелинейными данными | Хорошо подходит для линейно разделимых данных |

**Отличия от Наивных Байесовских Классификаторов:**

| Характеристика        | `GradientBoostingClassifier`                          | Наивный Байесовский Классификатор                                 |
|-----------------------|-------------------------------------------------------|-------------------------------------------------------------------|
| **Подход**            | Дискриминативный (хотя в основе лежит и генеративный) | Генеративный                                                      |
| **Предположения**     | Не делает предположений о независимости признаков     | Независимость признаков                                           |
| **Скорость обучения** | Может быть медленнее на больших данных                | Обычно быстрее                                                    |
| **Точность**          | Обычно более точный                                   | Может быть менее точным                                           |
| **Тип данных**        | Работает с различными типами данных                   | Особенно хорошо для текстовых данных (MultinomialNB, BernoulliNB) |

**Отличия от `LinearSVC`:**

| Характеристика    | `GradientBoostingClassifier`              | `LinearSVC`                                            |
|-------------------|-------------------------------------------|--------------------------------------------------------|
| **Тип модели**    | Ансамблевый метод, на основе деревьев     | Линейная модель, основанная на методе опорных векторов |
| **Подход**        | Нелинейный                                | Линейный                                               |
| **Обучение**      | Последовательное (бустинг)                | Оптимизация гиперплоскости                             |
| **Интерпретация** | Менее интерпретируема                     | Интерпретировать веса сложнее                          |
| **Точность**      | Часто обеспечивает более высокую точность | Хорошо работает для линейно разделимых данных          |


## Оценка точности
Для оценки точности была использована метрика `accuracy_score`.

`accuracy_score` — это метрика, используемая для оценки производительности моделей классификации. Она вычисляет долю правильно классифицированных объектов от общего числа объектов. Проще говоря, это процент правильно угаданных классов.

**Формула:**

$$ \text{Accuracy} = \frac{\text{Количество правильно классифицированных объектов}}{\text{Общее количество объектов}} $$

*   **Правильно классифицированные объекты (True Positives + True Negatives):** Это объекты, которые были правильно отнесены к своим классам. Например, SMS, которые были действительно спамом и были правильно классифицированы как спам, а также сообщения, которые были не спамом и были правильно классифицированы как не спам.
*   **Общее количество объектов:** Это общее количество всех объектов в тестовом наборе данных, включая и спам и не спам.


## Общая Сравнительная Таблица

| Характеристика               | BernoulliNB             | LogisticRegression                                 | LinearSVC              | RandomForestClassifier                           | GradientBoostingClassifier                              |
|------------------------------|-------------------------|----------------------------------------------------|------------------------|--------------------------------------------------|---------------------------------------------------------|
| **Тип**                      | Генеративный            | Дискриминативный                                   | Дискриминативный       | Ансамблевый (бэггинг)                            | Ансамблевый (бустинг)                                   |
| **Линейность**               | Линейный                | Линейный                                           | Линейный               | Нелинейный                                       | Нелинейный                                              |
| **Интерпретируемость**       | Высокая                 | Высокая                                            | Средняя                | Средняя                                          | Низкая                                                  |
| **Точность**                 | Низкая                  | Средняя                                            | Средняя - Высокая      | Высокая                                          | Очень Высокая                                           |
| **Вычислительная сложность** | Низкая                  | Средняя                                            | Средняя                | Средняя - Высокая                                | Высокая                                                 |
| **Выбросы**                  | Чувствителен            | Чувствителен                                       | Менее чувствителен     | Менее чувствителен                               | Устойчив                                                |
| **Предположения**            | Независимость признаков | Линейная разделимость, слабая корреляция признаков | Линейная разделимость  | Низкая корреляция признаков в отдельных деревьях | Не делает явных предположений о независимости признаков |
| **Вероятность**              | Предоставляет           | Предоставляет                                      | Нет                    | Предоставляет                                    | Предоставляет                                           |
| **Основные применения**      | Текстовая классификация | Бинарная классификация                             | Линейная классификация | Классификация различных типов данных             | Классификация различных типов данных                    |

Получив следующие результаты точности для каждой модели, обученной на задаче классификации SMS-сообщений на спам и не спам:

*   **MultinomialNB (Наивный Байес с мультиномиальным распределением):** 0.98386
*   **BernoulliNB (Наивный Байес с распределением Бернулли):** 0.97788
*   **Logistic Regression (Логистическая Регрессия):** 0.98446
*   **LinearSVC (Линейный Классификатор на основе метода опорных векторов):** 0.98565
*   **RandomForestClassifier (Классификатор на основе случайного леса):** 0.97549
*   **GradientBoostingClassifier (Классификатор на основе градиентного бустинга):** 0.97788

**Сравнение моделей:**

*   **Линейные модели (`Logistic Regression`, `LinearSVC`):** Показывают лучшие результаты, что указывает на то, что разделение спама и не спама хорошо описывается линейными моделями.
*  **Наивные байесовские классификаторы (`MultinomialNB`, `BernoulliNB`):** Показывают хорошие результаты, и могут быть использованы, как базовые модели, так и в случаях, когда скорость обучения является критичным фактором.
*   **Ансамблевые методы (`RandomForestClassifier`, `GradientBoostingClassifier`):** В данном случае, не показали каких-либо преимуществ, даже не смотря на их способность улавливать нелинейные зависимости, возможно, требуется более тонкая настройка.

**Выводы:**

*   **`LinearSVC`** — самый точный алгоритм в этом конкретном случае. Если точность является ключевым фактором, то следует выбрать `LinearSVC`.
*   **`Logistic Regression`** является хорошей альтернативой, которая немного уступает `LinearSVC` в точности, но при этом она более интерпретируема и предоставляет вероятности принадлежности к классу.
*   **`MultinomialNB`** также хорошо работает, предлагая разумный компромисс между точностью и скоростью обучения, что делает его полезным для задач с большими данными.
*   **`BernoulliNB`**, `RandomForestClassifier` и `GradientBoostingClassifier` показывают более низкие результаты и могут не являться оптимальным выбором.


In [35]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.metrics import accuracy_score

# Обучение модели и прогнозирование с оценкой качества модели
model_Multinomial = MultinomialNB()
model_Multinomial.fit(X_train_vec, y_train)
y_pred_Multinomial = model_Multinomial.predict(X_test_vec)
accuracy_Multinomial = accuracy_score(y_test, y_pred_Multinomial)

# Обучение модели и прогнозирование с оценкой качества модели
model_BernoulliNB = BernoulliNB()
model_BernoulliNB.fit(X_train_vec, y_train)
y_pred_BernoulliNB = model_BernoulliNB.predict(X_test_vec)
accuracy_BernoulliNB = accuracy_score(y_test, y_pred_BernoulliNB)

# Обучение модели и прогнозирование с оценкой качества модели
model_Logistic = LogisticRegression(solver='liblinear', random_state=52)
model_Logistic.fit(X_train_vec, y_train)
y_pred_Logistic = model_Logistic.predict(X_test_vec)
accuracy_Logistic = accuracy_score(y_test, y_pred_Logistic)

# Обучение модели и прогнозирование с оценкой качества модели
model_LinearSVC = LinearSVC(random_state=52)
model_LinearSVC.fit(X_train_vec, y_train)
y_pred_LinearSVC = model_LinearSVC.predict(X_test_vec)
accuracy_LinearSVC = accuracy_score(y_test, y_pred_LinearSVC)

# Обучение модели и прогнозирование с оценкой качества модели
model_RandomForest = RandomForestClassifier(random_state=52)
model_RandomForest.fit(X_train_vec, y_train)
y_pred_RandomForest = model_RandomForest.predict(X_test_vec)
accuracy_RandomForest = accuracy_score(y_test, y_pred_RandomForest)

# Обучение модели и прогнозирование с оценкой качества модели
model_GradientBoosting = GradientBoostingClassifier(random_state=52)
model_GradientBoosting.fit(X_train_vec, y_train)
y_pred_GradientBoosting = model_GradientBoosting.predict(X_test_vec)
accuracy_GradientBoosting = accuracy_score(y_test, y_pred_GradientBoosting)

# Создания массива моделей для виджетов
array_models = [model_Multinomial,model_BernoulliNB,model_Logistic, model_LinearSVC ,model_RandomForest,model_GradientBoosting]

# Данные для dataframe с информацией модели и её оценки 
data = {
    'Модель': ['MultinomialNB', 'BernoulliNB', 'Logistic Regression', 'LinearSVC', 'RandomForestClassifier', 'GradientBoostingClassifier'],
    'Точность': [accuracy_Multinomial, accuracy_BernoulliNB, accuracy_Logistic, accuracy_LinearSVC, accuracy_RandomForest, accuracy_GradientBoosting]
}

# Создание dataframe для вывода графиков 
df_models = pd.DataFrame(data)
df_models = df_models.sort_values(by='Точность')

# Создание гистограммы  
fig = px.bar(df_models, x='Модель', y='Точность',
             title='Сравнение точности моделей классификации SMS-спама',
             labels={'Точность': 'Точность', 'Модель': 'Модель классификации'},
             text_auto=".4f", # Чтобы можно было увидеть показатели без наведения
             range_y=[0.9, 1] # Диапазон чтобы график не растягивало 
             )
fig.show()

# Создание линейного графика  
fig_line = px.line(df_models, x='Модель', y='Точность',
             title='Сравнение точности моделей классификации SMS-спама',
             labels={'Точность': 'Точность', 'Модель': 'Модель классификации'},
             markers=True # Добавление маркеров на график для наглядности
             )
fig_line.show()

## Функция `classify_message` для проверки различных моделей классификации

Это функция, которая возвращает результат классификации (спам или не спам).

Для интерактивности использована библиотека `ipywidgets`.

`ipywidgets` — это библиотека Python, которая позволяет создавать интерактивные элементы управления (виджеты) в Jupyter Notebook и JupyterLab. Эти виджеты позволяют пользователю взаимодействовать с вашим кодом и данными, делая ваш анализ более динамичным и пользовательским.

**Основные возможности `ipywidgets`:**

*   **Интерактивность:** Добавляет интерактивные элементы управления в Jupyter Notebook.
*   **Разнообразие виджетов:** Предоставляет широкий спектр виджетов, таких, как слайдеры, кнопки, текстовые поля, выпадающие списки и т.д.
*   **Простота использования:** Легко интегрируется с кодом.
*   **Настраиваемость:** Позволяет настраивать внешний вид и поведение виджетов.



In [36]:
from ipywidgets import interact


def classify_message(message : str, model):
    # Преобразование текста сообщения в числовой формат
    message_vec = vectorizer.transform([message])
    # Использование модели для предсказания класса сообщения
    prediction = model.predict(message_vec)[0]
    print(f"Сообщение: \n\n{message}\n\nЯвляется ", 'спамом' if prediction == 1 else 'не спамом')
    
# Создание виджетов
interact(
    classify_message,
    message="Congratulations! You've won a $1,000 gift card. Claim now: [malicious link]",
    model=array_models
)

interactive(children=(Text(value="Congratulations! You've won a $1,000 gift card. Claim now: [malicious link]"…

<function __main__.classify_message(message: str, model)>

## Вывод

Шаблон являющийся спамом  `Congratulations! You've won a $1,000 gift card. Claim now: [malicious link]`
показал, что модели Логистической Регрессии и Градиентного Бустинга некорректно классифицируют данный текст как не спам.

Очевидно, что в ручную это сообщение легко идентифицируется как спам, однако указанные алгоритмы не справляются с задачей. Таким образом, их использование для классификации спама в данном случае является неэффективным. Эксперименты с другими моделями также выявили определенные ошибки, что свидетельствует о сложности задачи классификации.

Наиболее успешным в данном случае оказался Наивный байесовский классификатор, который лучше всего справился с идентификацией сообщений как спама. Это делает его предпочтительным выбором для решения данной задачи.